#### Importing Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isdir
from PIL import Image
from keras.models import load_model
from keras_facenet import FaceNet
from mtcnn import MTCNN

#### Data Pre processing 

In [2]:
def extract_face(filename, required_size = (160,160)):
    # Loading images
    image = Image.open(filename)
    # Converting it into RGB since the FaceNet model takes RGB images
    image = image.convert('RGB')
    # Converting the image to an array
    pixels = np.asarray(image)
    # Using the MTCNN detector with default weights
    detector = MTCNN()
    # Detecting the faces in the image
    result = detector.detect_faces(pixels)
    # Extracting the bounding box for the first detected face
    x1, y1, width, height = result[0]['box']
    # The values in result might be negative values. 
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1+width, y1+height
    # Extracting the face 
    face = pixels[y1:y2, x1:x2]
    # Resizing the face to model inputs
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = np.asarray(image)
    return face_array

In [3]:
def load_faces(directory):
    faces = []
    # Enumerating the files
    for file in listdir(directory):
        path = directory + file
        # Getting the faces
        face = extract_face(path)
        # Appending the face into the faces list.
        faces.append(face)
    return faces

In [13]:
def load_dataset(directory):
    X, y = [], []
    for subdir in listdir(directory):
        # path
        path = directory + subdir + '/'
        # ignoring if there are any files directly in the directory
        if not isdir(path):
            continue
        # Loading all the faces
        faces = load_faces(path)
        # creating labels
        labels = [subdir for _ in range(len(faces))]
        # Summary 
        print("Loaded faces = %d, %s", (len(faces), subdir))
        # Storing all the data
        X.extend(faces)
        print(type(X))
        y.extend(labels)
    return np.asarray(X), np.asarray(y)

In [14]:
# Loading training set
X_train, y_train = load_dataset('5-celebrity-faces-dataset/train/')
print(X_train.shape)
print(y_train.shape)
# Loading testing set
X_test, y_test = load_dataset('5-celebrity-faces-dataset/val/')
print(X_test.shape)
print(y_test.shape)

1/1 [==============================] - 0s 283ms/step
Loaded faces = %d, %s (14, 'ben_afflek')
<class 'list'>
1/1 [==============================] - 0s 152ms/step


1/1 [==============================] - 0s 145ms/step


1/1 [==============================] - 0s 344ms/step
Loaded faces = %d, %s (19, 'madonna')
<class 'list'>
1/1 [==============================] - 0s 196ms/step


1/1 [==============================] - 0s 343ms/step
Loaded faces = %d, %s (17, 'elton_john')
<class 'list'>
1/1 [==============================] - 0s 279ms/step


1/1 [==============================] - 0s 482ms/step
Loaded faces = %d, %s (22, 'mindy_kaling')
<class 'list'>
1/1 [==============================] - 0s 217ms/step


1/1 [==============================] - 0s 465ms/step
Loaded faces = %d, %s (21, 'jerry_seinfeld')
<class 'list'>
(93, 160, 160, 3)
(93,)
1/1 [==============================] - 0s 219ms/step


1/1 [==============================] - 0s 447ms/step
Loaded faces = %d, %s (5, 'ben_afflek')
<class 'list'>
1/1 [==============================] - 1s 523ms/step
Loaded faces = %d, %s (5, 'madonna')
<class 'list'>
1/1 [==============================] - 0s 497ms/step
Loaded faces = %d, %s (5, 'elton_john')
<class 'list'>
1/1 [==============================] - 0s 311ms/step


1/1 [==============================] - 0s 488ms/step
Loaded faces = %d, %s (5, 'mindy_kaling')
<class 'list'>
1/1 [==============================] - 1s 735ms/step
Loaded faces = %d, %s (5, 'jerry_seinfeld')
<class 'list'>
(25, 160, 160, 3)
(25,)


#### Creating Face Embeddings

In [15]:
model = FaceNet()

In [16]:
def get_embeddings(model, face_pixels):
    # Scaling the pixel values
    face_pixels = face_pixels.astype('float32')
    # Standardizing the face pixel values
    std, mean = face_pixels.std(), face_pixels.mean()
    face_pixels = (face_pixels - mean)/std
    # Transforming each face into a single sample
    samples = np.expand_dims(face_pixels, axis = 0)
    # Making predictions to get face embeddings
    face_embeddings = model.embeddings(samples)
    return face_embeddings

In [17]:
# Getting the face embeddings for training samples
X_train_embed = []
for face_pixels in X_train:
    embeddings  = get_embeddings(model, face_pixels)
    X_train_embed.append(embeddings)
X_train_embed = np.asarray(X_train_embed)

1/1 [==============================] - 0s 31ms/step


In [20]:
X_train_embed.shape

(93, 1, 512)

In [18]:
# Getting the face embeddings for training samples
X_test_embed = []
for face_pixels in X_test:
    embeddings  = get_embeddings(model, face_pixels)
    X_test_embed.append(embeddings)
X_test_embed = np.asarray(X_test_embed)

1/1 [==============================] - 0s 31ms/step
